In [1]:
import pandas as pd 
import os 
from sqlalchemy import create_engine

In [2]:
import pandas as pd
import sqlite3

# Load CSV
df = pd.read_csv("logistics_shipments_feature_engineered.csv")

# Create SQLite database
conn = sqlite3.connect("logistics.db")

# Write to SQLite table
df.to_sql("shipments", conn, if_exists="replace", index=False)

1932

In [3]:
pd.read_sql("SELECT COUNT(*) AS row_count FROM shipments", conn)

,row_count
0,1932


In [4]:
df.head(5)

,Shipment_ID,Origin_Warehouse,Destination,Carrier,Shipment_Date,Delivery_Date,Weight_kg,Cost,Status,Distance_miles,Transit_Days,Cost_per_mile,Cost_per_kg,Ship_Month,Route
0,SH10000,Warehouse_MIA,San Francisco,UPS,2023-10-02,2023-10-04,25.7,67.46,Delivered,291,2,0.231821,2.624903,10,Warehouse_MIA → San Francisco
1,SH10001,Warehouse_MIA,Atlanta,DHL,2023-12-06,2023-12-09,38.9,268.85,Delivered,1225,3,0.219469,6.911311,12,Warehouse_MIA → Atlanta
2,SH10002,Warehouse_LA,Houston,DHL,2023-09-18,2023-09-20,37.2,74.35,Delivered,220,2,0.337955,1.998656,9,Warehouse_LA → Houston
3,SH10003,Warehouse_BOS,Seattle,OnTrac,2023-01-26,2023-02-04,42.6,187.04,Delivered,1156,8,0.161799,4.390610,1,Warehouse_BOS → Seattle
4,SH10004,Warehouse_SF,Dallas,OnTrac,2023-06-03,2023-06-06,7.9,120.01,Delivered,1017,3,0.118004,15.191139,6,Warehouse_SF → Dallas


### Query 1 — Cheapest carrier overall

In [5]:
query = """
SELECT 
    Carrier,
    ROUND(AVG(Cost), 2) AS avg_cost
FROM shipments
GROUP BY Carrier
ORDER BY avg_cost ASC;
"""
pd.read_sql(query, conn)

,Carrier,avg_cost
0,USPS,181.92
1,OnTrac,185.14
2,LaserShip,188.28
3,Amazon Logistics,193.66
4,UPS,203.67
5,FedEx,218.42
6,DHL,221.47


### Query 2 — Most cost-efficient carrier (cost per mile)

In [6]:
query_2 = """
SELECT
    Carrier,
    ROUND(AVG(Cost_per_mile), 4) AS avg_cost_per_mile
FROM shipments
GROUP BY Carrier
ORDER BY avg_cost_per_mile ASC;
"""
pd.read_sql(query_2, conn)

,Carrier,avg_cost_per_mile
0,USPS,0.1589
1,Amazon Logistics,0.1712
2,OnTrac,0.1725
3,LaserShip,0.1731
4,UPS,0.1759
5,FedEx,0.1844
6,DHL,0.1882


### Query 3 — Cost vs speed trade-off

In [7]:
query_3 = """
SELECT
    Carrier,
    ROUND(AVG(Cost), 2) AS avg_cost,
    ROUND(AVG(Transit_Days), 2) AS avg_transit_days
FROM shipments
GROUP BY Carrier
ORDER BY avg_cost ASC;
"""
pd.read_sql(query_3, conn)

,Carrier,avg_cost,avg_transit_days
0,USPS,181.92,4.04
1,OnTrac,185.14,4.08
2,LaserShip,188.28,4.08
3,Amazon Logistics,193.66,4.27
4,UPS,203.67,4.19
5,FedEx,218.42,4.28
6,DHL,221.47,4.26


### Query 4 — Distance efficiency buckets

In [8]:
query_4 = """
SELECT
    CASE
        WHEN Distance_miles <= 300 THEN 'Short (0–300)'
        WHEN Distance_miles <= 700 THEN 'Medium (301–700)'
        WHEN Distance_miles <= 1200 THEN 'Long (701–1200)'
        ELSE 'Very Long (1200+)'
    END AS distance_bucket,
    ROUND(AVG(Cost_per_mile), 4) AS avg_cost_per_mile
FROM shipments
GROUP BY distance_bucket
ORDER BY avg_cost_per_mile DESC;
"""
pd.read_sql(query_4, conn)

,distance_bucket,avg_cost_per_mile
0,Short (0–300),0.3090
1,Medium (301–700),0.1966
2,Long (701–1200),0.1689
3,Very Long (1200+),0.1475


### Query 5 — Best carrier for long-distance shipments

In [9]:
query_5 = """
SELECT
    Carrier,
    ROUND(AVG(Cost_per_mile), 4) AS avg_cost_per_mile
FROM shipments
WHERE Distance_miles > 700
GROUP BY Carrier
ORDER BY avg_cost_per_mile ASC;
"""
pd.read_sql(query_5, conn)

,Carrier,avg_cost_per_mile
0,Amazon Logistics,0.1443
1,USPS,0.1461
2,OnTrac,0.1480
3,LaserShip,0.1484
4,UPS,0.1576
5,FedEx,0.1642
6,DHL,0.1680


### Query 6 — Warehouse performance

In [10]:
query_6 = """
SELECT
    Origin_Warehouse,
    COUNT(*) AS shipment_count,
    ROUND(AVG(Cost), 2) AS avg_cost
FROM shipments
GROUP BY Origin_Warehouse
ORDER BY shipment_count DESC;
"""
pd.read_sql(query_6, conn)

,Origin_Warehouse,shipment_count,avg_cost
0,Warehouse_LA,216,196.45
1,Warehouse_SF,212,200.65
2,Warehouse_HOU,203,204.45
3,Warehouse_ATL,203,199.01
4,Warehouse_MIA,192,200.57
5,Warehouse_CHI,188,186.24
6,Warehouse_BOS,187,205.05
7,Warehouse_SEA,184,209.25
8,Warehouse_DEN,184,191.71
9,Warehouse_NYC,163,193.50


### Query 7 — Most expensive routes

In [12]:
query_7 = """
SELECT
    Route,
    COUNT(*) AS shipments,
    ROUND(AVG(Cost), 2) AS avg_cost
FROM shipments
GROUP BY Route
HAVING shipments >= 5
ORDER BY avg_cost DESC
LIMIT 10;
"""
pd.read_sql(query_7, conn)

,Route,shipments,avg_cost
0,Warehouse_SEA → Boston,12,285.14
1,Warehouse_SEA → Miami,7,269.34
2,Warehouse_CHI → Phoenix,8,258.44
3,Warehouse_MIA → Boston,11,258.43
4,Warehouse_DEN → Boston,12,249.81
5,Warehouse_BOS → Boston,10,248.81
6,Warehouse_ATL → Boston,18,244.22
7,Warehouse_HOU → Seattle,11,242.85
8,Warehouse_BOS → Portland,17,242.65
9,Warehouse_LA → Atlanta,9,242.19


### Query 8 — Delivery reliability by carrier

In [13]:
query_8 = """
SELECT
    Carrier,
    ROUND(
        SUM(CASE WHEN Status = 'Delivered' THEN 1 ELSE 0 END) * 100.0 / COUNT(*),
        2
    ) AS delivery_rate_percent
FROM shipments
GROUP BY Carrier
ORDER BY delivery_rate_percent DESC;
"""
pd.read_sql(query_8, conn)

,Carrier,delivery_rate_percent
0,UPS,86.48
1,FedEx,82.59
2,OnTrac,82.41
3,DHL,81.48
4,USPS,81.29
5,LaserShip,81.10
6,Amazon Logistics,79.32


### Query 9 — Cost impact of delivery status

In [14]:
query_9 = """
SELECT
    Status,
    ROUND(AVG(Cost), 2) AS avg_cost,
    ROUND(AVG(Transit_Days), 2) AS avg_transit_days
FROM shipments
GROUP BY Status;
"""
pd.read_sql(query_9, conn)

,Status,avg_cost,avg_transit_days
0,Delayed,201.66,4.22
1,Delivered,200.53,4.19
2,In Transit,170.11,3.95
3,Lost,175.32,3.76
4,Returned,196.36,3.94


### Query 10 — Monthly seasonality analysis

In [15]:
query_10 = """
SELECT
    Ship_Month,
    COUNT(*) AS shipment_volume,
    ROUND(AVG(Cost), 2) AS avg_cost
FROM shipments
GROUP BY Ship_Month
ORDER BY Ship_Month;
"""
pd.read_sql(query_10, conn)

,Ship_Month,shipment_volume,avg_cost
0,1,158,187.98
1,2,136,188.01
2,3,160,189.03
3,4,158,187.71
4,5,169,200.40
5,6,157,200.49
6,7,160,203.48
7,8,174,186.75
8,9,153,203.62
9,10,170,191.16


In [16]:
conn.close()

## Key Insights from SQLite Analysis

From the SQLite-based analysis of the shipment data, several important logistics insights emerged. The results show that shipping costs vary significantly by carrier, with some carriers consistently offering lower average costs and better cost-per-mile efficiency, especially on long-distance shipments. Cost per mile tends to decrease as shipment distance increases, indicating economies of scale for longer routes. Warehouse-level analysis revealed that certain origin warehouses handle higher shipment volumes and also incur higher average costs, suggesting opportunities for operational optimization. Route-level queries highlighted a small set of routes that contribute disproportionately to total shipping costs, making them strong candidates for carrier renegotiation or alternative routing. Additionally, delivery performance analysis showed that the majority of shipments are successfully delivered, while delayed shipments are associated with higher average costs and longer transit times. Seasonal trends further indicate that shipment volume and costs fluctuate by month, reinforcing the importance of data-driven planning in logistics operations.